In [80]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
import torch
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Flatten, Dense, Dropout,Activation,MaxPooling2D,ReLU,Add,GlobalAvgPool2D
from tensorflow.keras import Model,Sequential
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping


2023-05-03 11:03:49.735771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Get the Data and Labels

In [2]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_bhat/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [3]:
data_dict = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [4]:
valence = np.load('data/data_prepared/labels/label_class_0.npy')
arousal = np.load('data/data_prepared/labels/label_class_1.npy')
dominance = np.load('data/data_prepared/labels/label_class_2.npy')
liking = np.load('data/data_prepared/labels/label_class_3.npy')

In [5]:
valence.shape

(1280,)

In [6]:
data_dict['sub01'].shape

(40, 40, 99)

## Create train test split

### Data

In [7]:
# training with 22 participants each has 40 videos = 880
train = np.zeros((0,40,99))
train[:] = np.nan
#train1 = 
test = np.zeros((0,40,99))
test[:] = np.nan

participation_counter = 0
for sub in data_dict.keys():
    if participation_counter < 22:
        train = np.concatenate((train, data_dict[sub]), axis=0)
    else:
        test = np.concatenate((test, data_dict[sub]), axis=0)
    participation_counter = participation_counter + 1

In [8]:
np.isnan(train).any()

False

In [9]:
np.isnan(test).any()

False

In [10]:
train.shape

(880, 40, 99)

In [11]:
test.shape

(400, 40, 99)

### Get generated Data

In [12]:
new_data = torch.load("data/psd_427")
new_data = new_data.numpy()

In [13]:
new_data.shape

(427, 3, 40, 104)

In [14]:
X_argument = np.zeros((880,40,99))
X_argument[:] = np.nan
counter = 0
for i in range(427):
    for j in range(3):
        if counter == 880:
            break
        else:
            X_argument[counter] = new_data[i][j][:,:99]
            counter = counter +1

In [15]:
np.isnan(X_argument).any()

False

In [16]:
# 2x
x2 = np.concatenate([X_argument, train], 0) 
print(x2.shape)

(1760, 40, 99)


----------------

In [17]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled_40-104_43_"+str(i)))
generated_data = np.concatenate(arrays, axis=0)

In [18]:
X_argument2 = np.zeros((880,40,99))
X_argument2[:] = np.nan
counter = 0
for i in range(430):
    for j in range(3):
        if counter == 880:
            break
        else:
            X_argument2[counter] = generated_data[i][j][:,:99]
            counter = counter +1

In [19]:
# 2x
x22 = np.concatenate([X_argument2, train], 0) 
print(x2.shape)

(1760, 40, 99)


In [20]:
np.array_equal(X_argument, train)

False

In [21]:
np.array_equal(X_argument2, train)

False

In [22]:
np.array_equal(X_argument, X_argument2)

False

In [23]:
np.array_equal(x2,x22)

False

### Labels

In [26]:
y_train_va = valence[: 880]
y_train_ar = arousal[: 880]
y_train_do = dominance[: 880]
y_train_li = liking[: 880]

y_test_va = valence[880: ]
y_test_ar = arousal[880: ]
y_test_do = dominance[880: ]
y_test_li = liking[880: ]

In [27]:
# Augmenting the labels 
# 2x
y_train_va2 = np.hstack((y_train_va,y_train_va))
y_train_ar2 = np.hstack((y_train_ar,y_train_ar))
y_train_do2 = np.hstack((y_train_do,y_train_do))
y_train_li2 = np.hstack((y_train_li,y_train_li))

In [28]:
print(y_train_va.shape)
print(y_train_ar.shape)
print(y_train_do.shape)
print(y_train_li.shape)
print(y_test_va.shape)
print(y_test_ar.shape)
print(y_test_do.shape)
print(y_test_li.shape)

(880,)
(880,)
(880,)
(880,)
(400,)
(400,)
(400,)
(400,)


## KNN

In [29]:
train.reshape(-1, 40*99).shape

(880, 3960)

In [60]:
# Define KNN based model for actual/original dataset
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  random_state=100
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtrain.shape,ytrain.shape)
  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval) 
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [61]:
# Define KNN based model for augmented data
def knnmodel2(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  #acc = round(acc, 4)
  print("Accuracy",acc)
  #print(classification_report(ytest, y_pred))
  #print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [62]:
trainlabel_list = [y_train_va, y_train_ar, y_train_do, y_train_li]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
score = []


for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel(train.reshape(-1, 40*99),traain,test.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append((sc,k))
    score.append(sc)
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

#for i,name in zip(scorelist,names):
#    print ("Accuracy of ",name ,i)

K value = 1
######################################### Valence #########################################
Accuracy 50.5
######################################### Arousal #########################################
Accuracy 48.0
######################################### Dominance #########################################
Accuracy 53.5
######################################### Liking #########################################
Accuracy 50.0
K value = 2
######################################### Valence #########################################
Accuracy 43.25
######################################### Arousal #########################################
Accuracy 47.5
######################################### Dominance #########################################
Accuracy 42.5
######################################### Liking #########################################
Accuracy 46.0
K value = 3
######################################### Valence #########################################
Accuracy 50.5
#######

Accuracy 59.0
K value = 20
######################################### Valence #########################################
Accuracy 50.0
######################################### Arousal #########################################
Accuracy 56.75
######################################### Dominance #########################################
Accuracy 52.0
######################################### Liking #########################################
Accuracy 57.75
K value = 21
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 53.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 58.5
K value = 22
######################################### Valence #########################################
A

Accuracy 60.25
K value = 39
######################################### Valence #########################################
Accuracy 52.75
######################################### Arousal #########################################
Accuracy 52.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 62.0
K value = 40
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 51.5
######################################### Dominance #########################################
Accuracy 55.00000000000001
######################################### Liking #########################################
Accuracy 61.5
K value = 41
######################################### Valence ##############################

In [63]:
score

[50.5,
 48.0,
 53.5,
 50.0,
 43.25,
 47.5,
 42.5,
 46.0,
 50.5,
 47.75,
 54.25,
 51.5,
 45.5,
 48.75,
 46.5,
 47.25,
 51.74999999999999,
 53.0,
 56.25,
 52.75,
 48.5,
 53.0,
 49.5,
 50.74999999999999,
 53.0,
 54.50000000000001,
 58.75,
 54.75,
 48.0,
 55.00000000000001,
 51.5,
 51.5,
 50.0,
 55.25,
 56.25,
 57.75,
 48.5,
 55.00000000000001,
 50.74999999999999,
 54.25,
 49.0,
 54.50000000000001,
 57.75,
 56.25,
 49.5,
 54.0,
 51.24999999999999,
 53.75,
 51.24999999999999,
 52.75,
 56.00000000000001,
 56.99999999999999,
 50.5,
 54.25,
 51.74999999999999,
 55.00000000000001,
 51.74999999999999,
 54.0,
 55.75,
 56.00000000000001,
 50.24999999999999,
 56.00000000000001,
 50.74999999999999,
 57.75,
 53.5,
 53.0,
 54.75,
 56.99999999999999,
 50.5,
 55.75,
 53.25,
 56.49999999999999,
 53.75,
 54.50000000000001,
 54.25,
 59.0,
 50.0,
 56.75,
 52.0,
 57.75,
 52.0,
 53.75,
 56.25,
 58.5,
 50.24999999999999,
 53.75,
 52.75,
 58.75,
 51.5,
 54.25,
 55.75,
 59.25,
 49.75,
 54.0,
 51.5,
 58.75,
 49.2

In [43]:
len(scorelist)

196

In [64]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(53.75, 19)
(56.75, 20)
(58.75, 7)
(63.0, 49)


### 2X Data

In [68]:
print("Augmented data x2 shape:",x2.shape)
print(y_train_va2.shape,y_train_ar2.shape,y_train_do2.shape,y_train_li2.shape)


Augmented data x2 shape: (1760, 40, 99)
(1760,) (1760,) (1760,) (1760,)


In [65]:
trainlabel_list = [y_train_va2, y_train_ar2, y_train_do2, y_train_li2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
scores = []

for k in range(1,50):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(x22.reshape(-1, 40*99),traain,test.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append((sc,k))
    scores.append(sc)
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 50.5
######################################### Arousal #########################################
Accuracy 48.0
######################################### Dominance #########################################
Accuracy 53.5
######################################### Liking #########################################
Accuracy 50.0
K value = 2
######################################### Valence #########################################
Accuracy 43.25
######################################### Arousal #########################################
Accuracy 47.5
######################################### Dominance #########################################
Accuracy 42.5
######################################### Liking #########################################
Accuracy 46.0
K value = 3
######################################### Valence #########################################
Accuracy 50.5
#######

Accuracy 54.25
######################################### Liking #########################################
Accuracy 59.0
K value = 20
######################################### Valence #########################################
Accuracy 50.0
######################################### Arousal #########################################
Accuracy 56.75
######################################### Dominance #########################################
Accuracy 52.0
######################################### Liking #########################################
Accuracy 57.75
K value = 21
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 53.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 58.5


Accuracy 53.75
######################################### Dominance #########################################
Accuracy 55.00000000000001
######################################### Liking #########################################
Accuracy 60.25
K value = 39
######################################### Valence #########################################
Accuracy 52.75
######################################### Arousal #########################################
Accuracy 52.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 62.0
K value = 40
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 51.5
######################################### Dominance ########################################

In [49]:
len(scorelist)

196

In [50]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(53.75, 19)
(56.75, 20)
(58.75, 7)
(63.0, 49)


In [ ]:
np.max(scores)

In [66]:
np.array_equal(score,scores)

True

In [ ]:
scorelist = [(15,0),(8,0),(88,0),(40,9),
             (5,6),(67,9),(7,6),(23,0),
             (12,2),(3,1),(5,3),(78,8)]
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

## SVM

In [71]:
# Define SVM based model for augmented data
def svmmodel(xtrain,ytrain,xtest,ytest,c,kernel = 'linear'):

  # SVM model

  model = SVC(C=c,kernel=kernel,gamma = 'auto')
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [74]:
# Using actual data
trainlabel_list = [y_train_va, y_train_ar, y_train_do, y_train_li]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(train.reshape(-1, 40*99),traain,test.reshape(-1, 40*99),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 56.75
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 72.75
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
A

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 56.00 <br>
Accuracy of  Dominance 73.0 <br>
Accuracy of  Liking 62.5 <br>

### 2X Data

In [76]:
# Using 2X data
trainlabel_list = [y_train_va2, y_train_ar2, y_train_do2, y_train_li2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
cval = [0.0001,0.001,0.01,0.1,1,10,100]

for c in cval:
  print("\n")
  print("C value =",c)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = svmmodel(x22.reshape(-1, 40*99),traain,test.reshape(-1, 40*99),tesst,c = c)
    scorelist.append(sc)



C value = 0.0001
######################################### Valence #########################################
Accuracy 59.25
######################################### Arousal #########################################
Accuracy 56.0
######################################### Dominance #########################################
Accuracy 73.0
######################################### Liking #########################################
Accuracy 62.5


C value = 0.001
######################################### Valence #########################################
Accuracy 55.5
######################################### Arousal #########################################
Accuracy 55.75
######################################### Dominance #########################################
Accuracy 73.25
######################################### Liking #########################################
Accuracy 62.5


C value = 0.01
######################################### Valence #########################################
A

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 60.00 <br>
Accuracy of  Dominance 73.25 <br>
Accuracy of  Liking 62.5 <br>

### Neural Network

In [77]:
# Split the data into training/testing sets
def deepmodel(modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size):
  # ()
  
  ytrain = to_categorical(ytrain)
  ytest = to_categorical(ytest)



  input_shape = (40, 99, 1)
  # print(xtrain.shape,xtest.shape)
  xtrain = np.reshape(xtrain,[-1, 40, 99, 1])
  xtest = np.reshape(xtest,[-1, 40, 99, 1])
  print("xtrain.shape,ytrain.shape")
  print(xtrain.shape,ytrain.shape)
  if modelnum == 1:
  # Model 1
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # model.add(Dense(1, activation='sigmoid'))
  elif modelnum == 2:
    # Resnet kind of model

    def conv_bn_relu(x, filters, kernel_size, strides):
        x = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same')(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x


    def identity_block(tensor, filters):
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=1)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        x = Add()([x, tensor])
        x = ReLU()(x)
        return x


    def projection_block(tensor, filters, strides):
        # left branch
        x = conv_bn_relu(tensor, filters=filters, kernel_size=1, strides=1)
        x = conv_bn_relu(x, filters=filters, kernel_size=3, strides=strides)
        x = Conv2D(filters=4*filters, kernel_size=1, strides=1)(x)  # notice: filters=4*filters
        x = BatchNormalization()(x)

        # right branch
        shortcut = Conv2D(filters=4*filters, kernel_size=1, strides=strides)(tensor)  # notice: filters=4*filters
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = ReLU()(x)
        return x


    def resnet_block(x, filters, reps, strides):
        x = projection_block(x, filters=filters, strides=strides)
        for _ in range(reps-1):  # the -1 is because the first block was a Conv one
            x = identity_block(x, filters=filters)
        return x


    input = Input(shape=(40, 99, 1))

    x = conv_bn_relu(input, filters=64, kernel_size=7, strides=2)  # [3]: 7x7, 64, strides 2
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)  # [3]: 3x3 max mool, strides 2

    x = resnet_block(x, filters=64, reps=3, strides=1)
    x = resnet_block(x, filters=128, reps=4, strides=2)  # s=2 ([2]: conv3_1)
    x = resnet_block(x, filters=256, reps=6, strides=2)  # s=2 ([2]: conv4_1)
    x = resnet_block(x, filters=512, reps=3, strides=2)  # s=2 ([2]: conv5_1)

    x = GlobalAvgPool2D()(x)  # [3]: average pool *it is not written any pool size so we use Global

    # # output = Dense(1, activation='sigmoid')(x)
    output = Dense(2, activation='softmax')(x)
    model = Model(input, output)



  model.compile(loss= 'categorical_crossentropy',#'binary_crossentropy','categorical_crossentropy'
                    optimizer=Adam(),
                    metrics=['accuracy'])
  # Fit data to model
  history = model.fit(xtrain,ytrain ,
            batch_size = batch_size,
            epochs=nepochs,
            verbose=1)

  # Generate generalization metrics
  scores = model.evaluate(xtest, ytest, verbose=0)#verbose =1 
  y_pred = model.predict(xtest)
  #     print("Fold number",fold_no)
  acc = scores[1]*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  print("Loss",scores[0])
  # print(classification_report(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  # print(confusion_matrix(ytest.argmax(axis=1), y_pred.argmax(axis=1)))
  return acc

## Normal Data

In [85]:
# Using original data
trainlabel_list = [y_train_va, y_train_ar, y_train_do, y_train_li]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 5 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,train.reshape(-1, 40,99),traain,test.reshape(-1, 40,99),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(880, 40, 99, 1) (880, 2)
Epoch 1/5
28/28 [==============================] - 3s 67ms/step - loss: 0.7121 - accuracy: 0.5261
Epoch 2/5
28/28 [==============================] - 2s 68ms/step - loss: 0.6920 - accuracy: 0.5250
Epoch 3/5
28/28 [==============================] - 2s 68ms/step - loss: 0.6754 - accuracy: 0.5784
Epoch 4/5
28/28 [==============================] - 2s 66ms/step - loss: 0.6735 - accuracy: 0.6011
Epoch 5/5
13/13 [==============================] - 0s 22ms/step
Accuracy 44.75
Loss 0.7484923005104065
######################################### Arousal #########################################
xtrain.shape,ytrain.shape
(880, 40, 99, 1) (880, 2)
Epoch 1/5
28/28 [==============================] - 3s 71ms/step - loss: 0.6924 - accuracy: 0.5795
Epoch 2/5
28/28 [==============================] - 2s 74ms/step - loss: 0.6902 - accuracy: 0.5830
Epoch 3/5
28/28 [====

Best results <br>
Accuracy of  Valence 44.75 <br>
Accuracy of  Arousal 54.00 <br>
Accuracy of  Dominance 54.00 <br>
Accuracy of  Liking 62.5 <br>

### 2x Data

In [86]:
# Using 2x data
trainlabel_list = [y_train_va2, y_train_ar2, y_train_do2, y_train_li2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
eps = 5 # epochs
bs = 32 # batch size
div = 2
# modelnum,xtrain,ytrain,xtest,ytest,nepochs,batch_size

for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
  print("#########################################",name,"#########################################")
  sc = deepmodel(1,x22.reshape(-1, 40,99),traain,test.reshape(-1, 40,99),tesst,nepochs=eps,batch_size=bs)
  scorelist.append(sc)

######################################### Valence #########################################
xtrain.shape,ytrain.shape
(1760, 40, 99, 1) (1760, 2)
Epoch 1/5
55/55 [==============================] - 5s 72ms/step - loss: 0.7064 - accuracy: 0.5318
Epoch 2/5
55/55 [==============================] - 4s 71ms/step - loss: 0.6911 - accuracy: 0.5352
Epoch 3/5
55/55 [==============================] - 4s 71ms/step - loss: 0.6902 - accuracy: 0.5352
Epoch 4/5
55/55 [==============================] - 4s 71ms/step - loss: 0.6883 - accuracy: 0.5460
Epoch 5/5
13/13 [==============================] - 0s 21ms/step
Accuracy 59.25
Loss 0.6817806363105774
######################################### Arousal #########################################
xtrain.shape,ytrain.shape
(1760, 40, 99, 1) (1760, 2)
Epoch 1/5
55/55 [==============================] - 5s 70ms/step - loss: 0.7123 - accuracy: 0.5676
Epoch 2/5
55/55 [==============================] - 4s 69ms/step - loss: 0.6832 - accuracy: 0.5830
Epoch 3/5
55/55 [

Best results <br>
Accuracy of  Valence 59.25 <br>
Accuracy of  Arousal 56.00 <br>
Accuracy of  Dominance 73.00 <br>
Accuracy of  Liking 62.5 <br>